Upom- I left the erros where I am not able to do the matrix multiplication in the middle of the notebook 

# NMF Modeling for player recomendations

non negative matrix factorization(NMF) is a feature reduction technique that decomposes a matrix into two smaller matricies that can be used to cluster features from the original matrix into topics as well as calculate the distance between samples in the original matrix to allow for recomendations between the closest samples to be created. I will be using NMF to generate player recomendations to find who has the cloest stats to  'Walker Zimmerman' and some of the other best players in the MLS 

In [1]:
## import packages and tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [2]:
## load data
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [3]:
## set index and drop negative and object columns 
X = df.set_index('player_name')
X = X.drop(columns= ['Base Salary','Player', 'Club',
                     'Nation', 'Position','fb_id', 'Squad_from_op','G-xG','np:G-xG'])
print(X.index[:4])
X.shape

Index(['Carles Gil', 'Julian Gressel', 'Emanuel Reynoso', 'Albert Rusnák'], dtype='object', name='player_name')


(705, 100)

In [4]:
# function to viz 
def display_features(H,W,feature_names, X_matrix ,no_top_features, no_top_players):
    # iterate through topics in topic-term matrix, 'H' aka
    # H is the hidden layer which is shape (F x C) feature times topic matrix
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([ (feature_names[i] + " (" + str(topic[i].round(2)) + ")")
          for i in topic.argsort()[:-no_top_features - 1:-1]]))
        top_player_indicies = np.argsort( W[:,topic_idx] )[::-1][0:no_top_players]
        for p_index in top_player_indicies:
            
            print(p_index," ",X_matrix.index[p_index])


In [5]:

c = 6
# Create a MaxAbsScaler: scaler
transformer = MaxAbsScaler().fit(X)

scaled_X= transformer.transform(X)

# Create an NMF model: nmf


nmf = NMF(n_components=c,max_iter=100000,init='nndsvda',alpha=.1, l1_ratio=.5, random_state=0)

W = nmf.fit_transform(scaled_X)
H = nmf.components_
err = nmf.reconstruction_err_

print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)

            
feature_names = X.columns.values
no_top_features = 3
no_top_players = 4

display_features(H,W,feature_names, X, no_top_features,no_top_players)

NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=6,
    random_state=0) error : 22.669488667905913 
  W shape (705, 6) H shape  (6, 100)
Topic 0:
Opposing_Poss (2.59) Opposing_Touches (2.58) Short_Cmp_pct (2.57)
601   Ventura Alvarado
688   Justin Portillo
609   Jordan Harvey
677   Mason Stajduhar
Topic 1:
prog_carry+lng_comp+crosses (1.43) carries_dist (1.28) carries (1.24)
696   Ilsinho
0   Carles Gil
2   Emanuel Reynoso
38   Alejandro Pozuelo
Topic 2:
touches_def_box (1.77) Total_Prg_Dist (1.7) Long_Att (1.69)
566   Brad Guzan
460   J.T. Marcinkowski
648   Maxime Crépeau
680   Andrew Tarbell
Topic 3:
pAdj_total_pressing (2.03) pAdj_Won_ball_5_sec (1.79) pAdj_pres_Mid_3rd (1.68)
513   Franco Ibarra
435   Judson
535   Chris Gloster
509   Matt LaGrassa
Topic 4:
Opposing_Poss (1.72) Opposing_Touches (1.71) Short_Cmp_pct (1.42)
470   Abu Danladi
699   Dante Sealy
45   Valentín Castellanos
130   Patryk Klimala
Topic 5:
recept_pct (1.61) Total_Cmp_pct (1.52) Aeria

In [17]:
 X.shape
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [6]:
## sparcity calculated

def calc_sparcity(X, c):
    n = X.shape[0]
    f = X.shape[1]
    
    sparcity = (c/f)+(c/n)
    
    print(sparcity)
    return sparcity

sp = calc_sparcity(X, c)

0.06851063829787234


In [15]:
X.head(2)

,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_OG,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches
player_name,,,,,,,,,,,,,,,,,,,,,
Carles Gil,49.1,20024,26.4,25.4,0.472441,5.275591,10.905512,48.818898,65.708661,74.3,...,0.0,10.963222,0.267396,0.401093,25.000000,2.406561,84.5,27.022169,24.960630,51.653543
Julian Gressel,54.0,20526,28.3,29.9,0.408027,3.210702,5.551839,37.324415,57.926421,64.4,...,0.0,13.905196,1.661976,1.495778,31.506849,7.589689,78.1,30.600462,17.959866,33.143813


## article inspo 

https://towardsdatascience.com/nmf-a-visual-explainer-and-python-implementation-7ecdd73491f8

https://towardsdatascience.com/topic-modeling-articles-with-nmf-8c6b2a227a45

https://towardsdatascience.com/improving-the-interpretation-of-topic-models-87fd2ee3847d

https://towardsdatascience.com/using-nmf-to-classify-companies-a77e176f276f

In [16]:
X.head(3)

,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_OG,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches
player_name,,,,,,,,,,,,,,,,,,,,,
Carles Gil,49.1,20024,26.4,25.4,0.472441,5.275591,10.905512,48.818898,65.708661,74.3,...,0.0,10.963222,0.267396,0.401093,25.000000,2.406561,84.5,27.022169,24.960630,51.653543
Julian Gressel,54.0,20526,28.3,29.9,0.408027,3.210702,5.551839,37.324415,57.926421,64.4,...,0.0,13.905196,1.661976,1.495778,31.506849,7.589689,78.1,30.600462,17.959866,33.143813
Emanuel Reynoso,49.9,20887,29.2,26.5,0.332075,3.433962,8.830189,42.113208,61.056604,69.0,...,0.0,12.162481,1.105680,0.982827,34.502924,6.695507,78.5,23.609394,18.792453,55.056604


In [111]:
#3 this looks like the actuall reconsittution of the matrix not the residuals between X and W*H
recont = X - W.dot(H)
recont.head(3)

,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_OG,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches
player_name,,,,,,,,,,,,,,,,,,,,,
Carles Gil,48.248025,20023.131773,26.180131,24.736799,0.088100,4.714876,10.439364,48.189695,64.963872,73.577792,...,-0.000696,10.603426,0.171716,0.296528,24.712747,2.280271,83.655033,26.743240,24.053467,50.981084
Julian Gressel,53.201417,20525.200833,28.055816,29.288255,0.230993,2.950725,5.287654,36.816885,57.338002,63.712209,...,-0.005939,13.504740,1.498855,1.327896,31.214356,7.292456,77.302398,30.297365,17.349057,32.811497


In [17]:
# The matrix multiplcation of W and H looks like residuals to the origianl X matrix 

print(W.dot(H).shape)
dot = pd.DataFrame(W.dot(H),index=X.index,columns = X.columns)
dot.head(3)

(705, 100)


,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_OG,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches
player_name,,,,,,,,,,,,,,,,,,,,,
Carles Gil,0.851975,0.868227,0.219869,0.663201,0.384341,0.560715,0.466148,0.629202,0.744789,0.722208,...,0.000696,0.359796,0.095679,0.104565,0.287253,0.126289,0.844967,0.278929,0.907163,0.672460
Julian Gressel,0.798583,0.799167,0.244184,0.611745,0.177034,0.259977,0.264185,0.507530,0.588419,0.687791,...,0.005939,0.400456,0.163121,0.167882,0.292493,0.297232,0.797602,0.303097,0.610810,0.332316
Emanuel Reynoso,0.964396,0.971956,0.260166,0.537434,0.374946,0.540096,0.438247,0.574787,0.683672,0.813828,...,0.000000,0.411698,0.114061,0.205993,0.357473,0.272911,0.895660,0.179066,0.790645,0.696183


So I created this function to reshape each row so I can get a forbenius norm from the row then add those values to the X matrix to use to sum up per cluster, but I am still unclear if the math above checks out or not. Please let me know what you think 

In [18]:
## this only works with 100 rows, would need reshape to be clean for selected row 

def fro_row(df):
    """ take each player and reshape row into 2d array to get forbeinus norm of array to show residual per player"""
    # create reconstituded matrix 
    recont = X - W.dot(H)
    print(recont.shape)
    # get difference between the two 
    resids = X - recont
    print(resids.shape)
    # create empty list
    
    to_row = np.zeros(len(resids))
    for i in range(len(resids)):
        # reshape row into 2d shape
        reshape = np.reshape(np.asarray(resids.iloc[i]),(4,25))
        # get forbeinus norm of row 
        fors = np.linalg.norm(reshape,'fro')
        # append to list 
        to_row[i]=fors
    
    # make column in df list or forbenius norms 
    df['resids']=to_row
    return df ,to_row

X,resids = fro_row(X)


(705, 100)
(705, 100)


In [19]:
X['resids'].sample(4)

player_name
Jonathan Bornstein    3.625649
Nathan Harriel        4.131226
Kamohelo Mokotjo      3.852211
Erik López            2.974997
Name: resids, dtype: float64

In [22]:
W.shape,H.shape

((705, 6), (6, 100))

In [24]:
def add_topics(X,W):
    df_new = X.copy()
    # Get the top predicted topic
    predicted_topics = [np.argsort(each)[::-1][0] for each in W]
    # Add to the df
    df_new['pred_topic_num'] = predicted_topics
    return df_new


df_new = add_topics(X,W)
df_new['pred_topic_num'].value_counts()

5    216
4    155
3    117
1    114
2     55
0     48
Name: pred_topic_num, dtype: int64

In [25]:
df_new['resids'] =resids

In [32]:
df_new.groupby('pred_topic_num').agg('mean')

,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches,resids
pred_topic_num,,,,,,,,,,,,,,,,,,,,,
0,50.272917,20308.437500,29.218050,2.516667,0.027713,0.394309,1.340241,20.145032,24.929155,80.402083,...,6.644179,0.700223,0.510491,38.157076,2.595816,89.203125,19.872084,4.868652,7.869659,2.902597
1,49.355263,20054.921053,27.213158,14.826316,0.171247,1.721718,4.735134,38.953701,49.510883,78.241228,...,11.456879,1.151248,1.180046,36.093369,6.033405,84.890789,16.002751,13.272666,33.102439,3.727553
2,50.314545,20359.800000,20.342864,15.820000,0.002315,0.023348,0.003266,21.807596,30.736663,70.503636,...,8.302960,0.002203,0.000000,21.100440,0.037597,97.964545,55.299469,8.138940,0.025543,3.217983
3,50.465812,20282.931624,29.368376,11.611111,0.072660,0.803332,3.440994,35.687884,44.124427,80.293162,...,13.837470,1.500493,1.870303,39.143828,9.317676,86.101282,15.945393,9.530859,19.821640,3.748520
4,49.944516,20207.090323,24.760645,12.919355,0.137118,1.119038,1.934766,19.255394,25.738873,74.683871,...,8.003120,2.755589,0.731037,28.474416,3.604215,79.209355,10.686611,6.662305,28.283309,3.222569
5,49.908333,20264.773148,32.466204,18.323611,0.039693,0.443821,3.223775,43.144869,50.854137,84.656481,...,13.213986,3.134004,1.222380,39.719735,8.164238,90.540278,20.890420,10.668618,10.599520,3.781718


In [157]:
resids[:4]

array([4.41871794, 3.67485108, 4.59674599, 3.61292885])

In [30]:
rec_psum = recont.sum(axis=1)
rec_psum[:3]

player_name
Carles Gil         23581.230738
Julian Gressel     23378.077065
Emanuel Reynoso    24004.577704
dtype: float64

In [36]:
rec_psum.shape

(705,)

In [38]:
p_sum = X.sum(axis=1)
p_sum.shape

(705,)

In [42]:
resids = p_sum - rec_psum
resids[:3]

player_name
Carles Gil         34.747151
Julian Gressel     30.064566
Emanuel Reynoso    38.059811
dtype: float64

In [44]:
np.mean(resids)

26.319020420772862

In [45]:
resids= pd.DataFrame(W.dot(H),index=df.player_name)
resids = resids.sum(axis=1)
resids

player_name
Carles Gil             34.747151
Julian Gressel         30.064566
Emanuel Reynoso        38.059811
Albert Rusnák          28.620149
Maximiliano Moralez    33.684569
                         ...    
Jordan Morris          20.486573
Jayden Nelson          32.188292
Jordy Delem            15.394986
Daniel Aguirre         18.737339
Kamron Habibullah      17.471507
Length: 705, dtype: float64

In [51]:
W@H

array([[0.85197459, 0.86822661, 0.21986889, ..., 0.27892897, 0.90716261,
        0.67245961],
       [0.79858294, 0.79916715, 0.24418375, ..., 0.30309651, 0.61080961,
        0.33231565],
       [0.96439593, 0.97195632, 0.26016575, ..., 0.17906613, 0.79064502,
        0.69618303],
       ...,
       [0.88521955, 0.87299031, 0.26399543, ..., 0.15549357, 0.05220842,
        0.03651644],
       [0.88836726, 0.88254906, 0.27709378, ..., 0.13518832, 0.14778125,
        0.10026716],
       [0.94801089, 0.93950444, 0.27103235, ..., 0.15833108, 0.12581414,
        0.13034317]])

In [132]:
resids_n =np.linalg.norm(X - W@H,'fro') 
resids_n

538312.8145058294

In [133]:
np.linalg.norm(resids,'fro')

532616.6586978853

In [20]:
W.dot(H).shape

(705, 100)

In [23]:
W.T

array([[0.00486865, 0.00582262, 0.        , ..., 0.26004752, 0.19895114,
        0.2624814 ],
       [0.50314099, 0.2103056 , 0.45755273, ..., 0.        , 0.01838062,
        0.06130747],
       [0.13435977, 0.15306877, 0.04893328, ..., 0.00068406, 0.        ,
        0.        ],
       [0.00246018, 0.10262688, 0.14257729, ..., 0.14765028, 0.06226103,
        0.10758926],
       [0.14102897, 0.08830592, 0.19470521, ..., 0.00164532, 0.07820347,
        0.04687689],
       [0.01544476, 0.13170514, 0.        , ..., 0.01977138, 0.10528649,
        0.        ]])

In [22]:
H.dot(W.T).shape

ValueError: shapes (6,100) and (6,705) not aligned: 100 (dim 1) != 6 (dim 0)

In [9]:
## why c

# Get the residuals for each document
r = np.zeros(X.shape[0])
for row in range(X.shape[0]):
    r[row] = np.linalg.norm(X.iloc[row, :] - H[row, :].dot(W), 'fro')
# Add the residuals to the df
df['resid'] = r

ValueError: shapes (100,) and (705,6) not aligned: 100 (dim 0) != 705 (dim 0)

In [14]:
W[row:6]

array([[0.00486865, 0.50314099, 0.13435977, 0.00246018, 0.14102897,
        0.01544476],
       [0.00582262, 0.2103056 , 0.15306877, 0.10262688, 0.08830592,
        0.13170514],
       [0.        , 0.45755273, 0.04893328, 0.14257729, 0.19470521,
        0.        ],
       [0.06501044, 0.20905528, 0.1221684 , 0.01971761, 0.16303824,
        0.09735363],
       [0.        , 0.27920385, 0.13103677, 0.13248667, 0.17015382,
        0.06048928],
       [0.        , 0.34581426, 0.07319704, 0.14844051, 0.20053544,
        0.01511073]])

In [16]:
Multiply a row of the weight matrix to the H matrix to get the approxamations for that player

W[row,:].dot(H)

array([8.51974595e-01, 8.68226611e-01, 2.19868893e-01, 6.63201194e-01,
       3.84340636e-01, 5.60714692e-01, 4.66148064e-01, 6.29202235e-01,
       7.44789327e-01, 7.22207731e-01, 5.79920169e-01, 6.14124265e-01,
       5.04111928e-01, 5.36866799e-01, 8.72339463e-01, 4.75518740e-01,
       5.41174506e-01, 8.17593718e-01, 4.36665792e-01, 4.82461893e-01,
       4.44764789e-01, 7.23255711e-01, 2.39451800e-01, 2.25321109e-01,
       4.86168772e-01, 6.51206821e-01, 3.43810779e-01, 6.74399221e-01,
       6.83527255e-01, 4.97984479e-01, 6.11998827e-01, 6.24147458e-01,
       2.50148821e-01, 7.49763939e-01, 7.60685623e-01, 6.71573042e-01,
       5.38196347e-01, 4.99161542e-01, 3.61189823e-01, 3.53384957e-01,
       5.33111436e-01, 7.52128791e-01, 7.18693918e-01, 8.85171154e-01,
       3.88408688e-01, 1.63563369e-01, 3.80433790e-01, 6.75235379e-02,
       2.33224818e-01, 1.53460751e-01, 3.48781084e-01, 1.46046311e-01,
       3.00593498e-01, 2.61520899e-02, 2.55439257e-02, 2.01117385e-02,
      

In [20]:
## inital row 
scaled_X[row, :]

array([0.89110708, 0.89329051, 0.264     , 0.74705882, 0.61780739,
       1.        , 0.64150069, 0.57128497, 0.70174299, 0.743     ,
       0.58177837, 0.62877584, 0.3496063 , 0.37958699, 0.869     ,
       0.49933818, 0.57313032, 0.821     , 0.57390952, 0.59186352,
       0.541     , 0.67068497, 0.00695936, 0.09958314, 0.51017861,
       0.60811629, 0.31552306, 0.62063586, 0.59701912, 0.38582677,
       0.673     , 0.52755906, 0.31496063, 0.67397825, 0.82956566,
       0.64255403, 0.51743532, 0.54133858, 0.38057743, 0.28121485,
       0.34645669, 0.7145881 , 0.65784655, 0.851     , 0.55463799,
       0.        , 0.42519685, 0.        , 0.174     , 0.15223097,
       0.32789651, 0.16456693, 0.358     , 0.00960134, 0.00685239,
       0.00472441, 0.00629921, 0.02975521, 0.34370079, 0.08836395,
       0.07952756, 0.16394826, 0.14229471, 0.00885827, 0.0542993 ,
       0.02533967, 0.01053713, 0.04327091, 0.08213372, 0.03800951,
       0.18584834, 0.34854206, 0.27614895, 0.15107787, 0.26846

In [21]:
scaled_X[row, :] - W[row,:].dot(H)

array([ 3.91324832e-02,  2.50638959e-02,  4.41311069e-02,  8.38576297e-02,
        2.33466754e-01,  4.39285308e-01,  1.75352630e-01, -5.79172629e-02,
       -4.30463407e-02,  2.07922688e-02,  1.85819767e-03,  1.46515786e-02,
       -1.54505629e-01, -1.57279813e-01, -3.33946293e-03,  2.38194352e-02,
        3.19558148e-02,  3.40628174e-03,  1.37243732e-01,  1.09401625e-01,
        9.62352109e-02, -5.25707414e-02, -2.32492443e-01, -1.25737969e-01,
        2.40098333e-02, -4.30905283e-02, -2.82877195e-02, -5.37633570e-02,
       -8.65081328e-02, -1.12157707e-01,  6.10011730e-02, -9.65884033e-02,
        6.48118090e-02, -7.57856864e-02,  6.88800359e-02, -2.90190150e-02,
       -2.07610266e-02,  4.21770407e-02,  1.93876050e-02, -7.21701093e-02,
       -1.86654743e-01, -3.75406949e-02, -6.08473699e-02, -3.41711538e-02,
        1.66229299e-01, -1.63563369e-01,  4.47630601e-02, -6.75235379e-02,
       -5.92248178e-02, -1.22977999e-03, -2.08845713e-02,  1.85206186e-02,
        5.74065022e-02, -

In [23]:
(scaled_X[row, :] - W[row,:].dot(H)).shape

(100,)

In [22]:
## I need to be able to use the froberious norm but the shape has to be 2D and i only have one 

np.linalg.norm(scaled_X[row, :] - W[row,:].dot(H), 'fro')

ValueError: Invalid norm order 'fro' for vectors

 from numpy  https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html i need 2 dims but I only have 1 , is there a way to get the error per row ?

The Frobenius norm is given by [1]:

The nuclear norm is the sum of the singular values.

Both the Frobenius and nuclear norm orders are only defined for matrices and raise a ValueError when x.ndim != 2.

In [ ]:
other resource about norm 

https://blog.acolyer.org/2019/02/18/the-why-and-how-of-nonnegative-matrix-factorization/
    
    
    

In [11]:
print(H.shape)
print(W.shape)

(6, 100)
(705, 6)


In [10]:
row =1
H[row, :].dot(W)

ValueError: shapes (100,) and (705,6) not aligned: 100 (dim 0) != 705 (dim 0)

ValueError: shapes (705,6) and (100,) not aligned: 6 (dim 1) != 100 (dim 0)

In [6]:
def add_topics(X,W):
    df_new = X.copy()
    # Get the top predicted topic
    predicted_topics = [np.argsort(each)[::-1][0] for each in W]
    # Add to the df
    df_new['pred_topic_num'] = predicted_topics
    return df_new


df_new = add_topics(X,W)
df_new['pred_topic_num'].describe()

count    705.00000
mean       3.22695
std        1.65389
min        0.00000
25%        2.00000
50%        4.00000
75%        5.00000
max        5.00000
Name: pred_topic_num, dtype: float64

In [7]:
df_new['pred_topic_num'].value_counts()

5    216
4    155
3    117
1    114
2     55
0     48
Name: pred_topic_num, dtype: int64

In [255]:
df_new[df_new['pred_topic_num']==8]

,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches,pred_topic_num
player_name,,,,,,,,,,,,,,,,,,,,,
Zico Bailey,51.0,20501,33.6,7.6,0.118421,1.052632,2.894737,43.289474,49.210526,88.0,...,12.656635,0.654654,1.309307,36.842105,7.419407,92.20,11.229947,7.631579,14.736842,8
Jarosław Niezgoda,55.1,22416,30.0,2.9,0.241379,2.068966,2.413793,16.206897,21.724138,74.6,...,7.322356,3.661178,0.000000,33.333333,2.092102,73.75,7.936508,4.482759,21.034483,8
Eddie Munjoma,48.9,20180,25.7,5.2,0.076923,0.961538,3.653846,37.500000,48.461538,77.4,...,11.664252,2.268049,1.620035,38.709677,7.128154,84.10,19.047619,11.730769,19.423077,8
McKinze Gaines,46.0,18825,22.7,2.4,0.083333,0.833333,0.833333,10.833333,12.916667,83.9,...,6.020363,2.257636,0.752545,18.181818,2.257636,79.75,6.451613,3.750000,15.416667,8
Ralph Priso-Mbongue,50.6,21171,32.3,6.3,0.015873,0.317460,3.809524,42.380952,50.793651,83.4,...,11.981049,2.549159,1.784412,34.482759,10.706470,89.20,13.437500,7.777778,14.444444,8
Justin Morrow,50.6,21171,32.5,10.9,0.018349,0.091743,2.935780,36.055046,43.119266,83.6,...,8.545531,2.210051,1.178694,44.736842,5.598796,88.55,8.723404,5.688073,14.311927,8
Christian Ramirez,54.9,21209,12.9,1.0,0.000000,1.000000,2.000000,17.000000,24.000000,70.8,...,6.412372,3.206186,0.000000,12.900000,1.603093,74.15,8.333333,5.000000,15.000000,8
Matko Miljevic,47.1,19792,34.8,1.6,0.250000,0.625000,3.750000,42.500000,51.250000,82.9,...,7.515663,1.073666,0.000000,50.000000,6.441997,86.20,12.195122,13.125000,35.000000,8
Tony Rocha,46.5,19332,18.2,0.3,0.000000,1.000000,3.000000,18.000000,22.000000,81.8,...,14.069936,0.000000,3.517484,33.333333,0.000000,84.35,9.090909,2.000000,24.000000,8


In [247]:
df_new[df_new['pred_topic_num']==5]

,Opposing_Poss,Opposing_Touches,Tackle_pct_won,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,...,pAdj_Recoveries,pAdj_Aerial_Duels_lost,lost_tackles,True_tackle_pct,pAdj_Tackle_int_blocks,avg_shrt+med_pass_pct,pct_long_balls,prog_carry+lng_comp+crosses,attacking_touches,pred_topic_num
player_name,,,,,,,,,,,,,,,,,,,,,
Albert Rusnák,49.1,19279,28.7,33.8,0.213018,2.573964,4.289941,40.739645,48.727811,83.6,...,10.591897,0.208707,0.417415,32.876712,3.287140,91.15,24.833030,15.355030,30.857988,5
Javier Eduardo López,47.4,18641,23.9,25.0,0.224000,1.960000,4.440000,30.760000,38.640000,79.6,...,10.141087,0.364787,0.510702,25.000000,2.699426,86.95,18.944099,10.360000,27.360000,5
Cristián Dajome,53.9,21193,27.9,29.7,0.175084,1.178451,3.164983,23.872054,33.131313,72.1,...,8.588694,1.296407,1.350424,35.403727,5.617762,77.20,15.752033,11.885522,30.336700,5
Ryan Hollingshead,48.9,20180,26.8,29.6,0.097973,1.013514,3.952703,41.081081,50.202703,81.8,...,12.579353,3.130608,1.764525,34.972678,7.741141,87.60,12.651413,10.912162,21.486486,5
Matías Vera,54.9,21209,25.3,27.2,0.062500,0.992647,3.786765,34.852941,39.044118,89.3,...,13.201943,1.119808,1.237682,34.567901,6.954595,91.70,18.267420,8.897059,14.264706,5
George Bello,42.6,17449,33.1,27.1,0.077491,0.811808,1.808118,40.000000,45.977860,87.0,...,11.935680,1.438034,1.150427,46.987952,7.118267,90.25,6.982343,8.007380,23.062731,5


In [248]:
jm = normed_df.loc['Albert Rusnák']
sims = normed_df.dot(jm)

sims.nlargest(10)

Albert Rusnák           1.000000
Diego Fagúndez          0.968353
Yimmi Chará             0.951997
Víctor Vázquez          0.931466
Ruan                    0.927384
Randall Leal            0.916339
Javier Eduardo López    0.915911
Nani                    0.915547
Djordje Mihailovic      0.912802
Maximiliano Moralez     0.910349
dtype: float64

In [ ]:
Having both creative players Albert Rusnak and Chofis be in the smallest group 

 wow , these groups actually have players with really simular skills in the same groups, 
need to tune NMF to get best amount of groups. 

In [243]:
## this is double scaled so not good 

no_top_features = 5
no_top_players = 4
errors = {}
components_dict = {}
for c in range(4,12):
    
    c = c
    # Create a MaxAbsScaler: scaler
    transformer = MaxAbsScaler().fit(X)

    scaled_X= transformer.transform(X)
    
    # Create an NMF model: nmf
    
#     transformer = Normalizer().fit(scaled_X)
#     normed_X= transformer.transform(scaled_X)
    
    nmf = NMF(n_components=c,max_iter=100000,init='nndsvda',alpha=.1, l1_ratio=.5, random_state=0)
   


    W = nmf.fit_transform(scaled_X)
    H = nmf.components_
    err = nmf.reconstruction_err_
    print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)
    
    feature_names = X.columns.values
    p_names = X.index.values
  
   

    display_features(H,W,feature_names, X, no_top_features,no_top_players)
    errors[c]=err
    # Create a DataFrame: components_df
    components_df = pd.DataFrame(nmf.components_,columns=X.columns)
    # put data frame in dictionary to observe 
    components_dict[c]=components_df


NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=4,
    random_state=0) error : 24.978212706168456 
  W shape (705, 4) H shape  (4, 100)
Topic 0:
recept_pct (1.72) Opposing_Poss (1.69) Opposing_Touches (1.66) Total_Cmp_pct (1.65) Short_Cmp_pct (1.6)
435   Judson
513   Franco Ibarra
535   Chris Gloster
455   Nathan Cardoso
Topic 1:
Opposing_Poss (2.42) Opposing_Touches (2.4) Short_Cmp_pct (2.13) avg_shrt+med_pass_pct (2.02) Medium_Cmp_pct (1.92)
470   Abu Danladi
699   Dante Sealy
631   Héber
260   Erik Hurtado
Topic 2:
Medium_Cmp_pct (2.56) avg_shrt+med_pass_pct (2.55) recept_pct (2.55) Short_Cmp_pct (2.54) Opposing_Poss (2.45)
663   Zac MacMath
642   Jonathan Bond
643   Alex Bono
668   David Ochoa
Topic 3:
prog_carry+lng_comp+crosses (1.71) Nineties (1.59) Total_Att (1.44) carries_dist (1.42) carries (1.36)
696   Ilsinho
0   Carles Gil
581   Gedion Zelalem
2   Emanuel Reynoso
NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=5,
    ra

# looking for coherent groups of features inside of components to pick c  

In [23]:
for i in components_dict:
    print('This is nmf with :',i, ' components')
    print(' error = ',errors[i])
    for a in range(len(components_dict[i])):
        component = components_dict[i].iloc[a]
        print(component.nlargest())
    print('\n')

This is nmf with : 4  components
 error =  4.596288813307577
avg_shrt+med_pass_pct          4.454749
pct_long_balls                 2.280901
prog_carry+lng_comp+crosses    1.288301
pAdj_Recoveries                0.976971
pAdj_PK_conceded               0.070610
Name: 0, dtype: float64
pAdj_def_interceptions         2.668022
pAdj_Tackle_int_blocks         2.411153
pAdj_Recoveries                1.561454
prog_carry+lng_comp+crosses    1.371265
pAdj_from_mish_tackles_won     1.288471
Name: 1, dtype: float64
avg_shrt+med_pass_pct          2.364094
attacking_touches              2.257160
prog_carry+lng_comp+crosses    1.444761
pAdj_Recoveries                0.724509
lost_tackles                   0.417975
Name: 2, dtype: float64
avg_shrt+med_pass_pct     1.170728
pAdj_Aerial_Duels_lost    1.055999
pAdj_Recoveries           0.230715
attacking_touches         0.211670
pAdj_Tackle_int_blocks    0.185520
Name: 3, dtype: float64


This is nmf with : 5  components
 error =  3.672639771965719
avg_s

In [249]:
transformer = Normalizer().fit(W)
normed_X= transformer.transform(W)
# can add back club and salary here 
normed_df = pd.DataFrame(normed_X,index=df.player_name.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 705 entries, Carles Gil to Kamron Habibullah
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       705 non-null    float64
 1   1       705 non-null    float64
 2   2       705 non-null    float64
 3   3       705 non-null    float64
 4   4       705 non-null    float64
 5   5       705 non-null    float64
 6   6       705 non-null    float64
 7   7       705 non-null    float64
 8   8       705 non-null    float64
 9   9       705 non-null    float64
 10  10      705 non-null    float64
dtypes: float64(11)
memory usage: 66.1+ KB


In [259]:
H.shape

(11, 100)

array([2.71230400e+00, 2.69568535e+00, 7.73758645e-01, 5.46327574e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.10300466e-01,
       2.84230726e-01, 2.43767940e+00, 1.38574997e-01, 0.00000000e+00,
       3.19043583e-01, 3.10743865e-01, 2.66725724e+00, 2.49169594e-01,
       2.52335009e-01, 2.53603143e+00, 0.00000000e+00, 0.00000000e+00,
       1.71713352e+00, 2.20329696e-01, 1.41241154e-01, 1.27488573e-01,
       1.18126615e-02, 0.00000000e+00, 0.00000000e+00, 1.65582378e-01,
       0.00000000e+00, 0.00000000e+00, 1.91706666e+00, 0.00000000e+00,
       1.09983832e-02, 1.15047106e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.25686146e-02,
       0.00000000e+00, 1.67478014e-01, 1.09748515e-01, 2.40320076e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.19232914e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.60830867e-02, 0.00000000e+00, 0.00000000e+00,
      

In [258]:
normed_df.loc['Walker Zimmerman'].nlargest(10)

2     0.752348
7     0.435393
5     0.292851
0     0.247659
9     0.166075
10    0.157548
8     0.151460
6     0.148214
1     0.000000
3     0.000000
Name: Walker Zimmerman, dtype: float64

In [267]:
df[df['player_name']=='Walker Zimmerman'].index

Int64Index([376], dtype='int64')

In [ ]:
walker is in group 2 


In [269]:
H.shape

(11, 100)

In [250]:
normed_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10
Carles Gil,0.0,0.353417,0.000000,0.722519,0.000000,0.283494,0.495796,0.117235,0.070935,0.090020,0.000000
Julian Gressel,0.0,0.000000,0.200826,0.747907,0.034024,0.421695,0.265830,0.245432,0.000000,0.097122,0.284578
Emanuel Reynoso,0.0,0.599706,0.000000,0.552760,0.193321,0.261981,0.376316,0.027089,0.000000,0.261118,0.135174


In [251]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson        1.000000
D.J. Taylor           0.961790
Emmanuel Más          0.961358
Justin Che            0.950543
Ronald Hernández      0.945051
Cristián Gutiérrez    0.944265
Marco Farfan          0.941027
Zorhan Bassong        0.940084
Milton Valenzuela     0.939209
Christian Mafla       0.936941
dtype: float64

In [38]:
print([i for i in df['player_name'] if 'Walker' in i ])

['Walker Zimmerman']


In [252]:
wz = normed_df.loc['Walker Zimmerman']
sims = normed_df.dot(wz)

sims.nlargest(10)

Walker Zimmerman      1.000000
Tanner Beason         0.986032
Ranko Veselinović     0.969065
Bill Tuiloma          0.967479
Xavier Arreaga        0.964666
Robin Jansson         0.964577
Jonathan Mensah       0.961700
Gustavo Vallecilla    0.958508
Maxime Chanot         0.958344
Antonio Carlos        0.957964
dtype: float64

In [40]:
jm = normed_df.loc['Jonathan Mensah']
sims = normed_df.dot(jm)

sims.nlargest(10)

Jonathan Mensah      1.000000
Antonio Carlos       0.996214
Rudy Camacho         0.994820
Andy Rose            0.993489
Larrys Mabiala       0.993116
Julio Cascante       0.993039
Josh Williams        0.989379
Nkosi Burgess        0.988423
Matt Hedges          0.986246
Ranko Veselinović    0.984544
dtype: float64

## need to test results with less features to see if error goes down 